In [ ]:
import os
import sys
import glob
import numpy as np
import obspy

localLibPath = os.path.join('..', 'lib')
sys.path.append(localLibPath)
import seisan_classes
import libSeisan2Pandas as seisan
import libMVO 
from SAM import DSAM
from ASL import montserrat_topo_map, Grid, ASL, simulate_DSAM, plot_DSAM

SDS_DIR = '/data/SDS' #'/Volumes/NTFS_2TB_EXT/SDS'
HOME = os.path.expanduser('~')
DATA_DIR = os.path.join(HOME, 'Dropbox/BRIEFCASE/MESS2024/skience2024_GTplus/02 Volcano Monitoring/data')
#SAM_DIR = os.path.join(DATA_DIR,'continuous','SAM')
SAM_DIR = '/data/SAM' #os.path.join(DATA_DIR,'continuous','SAM')
DEM_DIR = os.path.join(DATA_DIR,'DEMs')
RESPONSE_DIR = os.path.join(DATA_DIR,'responses')
SEISAN_DATA = os.path.join( '/data', 'SEISAN_DB')
DB = 'MVOE_'

dome_location = {'lat':16.71111, 'lon':-62.17722}

def initial_source(lat=dome_location.lat, lon=dome_location.lon):
    return {'lat':lat, 'lon':lon}

def make_grid(center_lat=dome_location.lat, center_lon=dome_location.lon, node_spacing_m = 100, grid_size_lat_m = 10000, grid_size_lon_m = 8000)
    nlat = int(grid_size_lat_m/node_spacing_m) + 1
    nlon = int(grid_size_lon_m/node_spacing_m) + 1
    return Grid(center_lat, center_lon, nlat, nlon, node_spacing_m)  

def asl_event(st, Q=23, surfaceWaveSpeed_kms = 1.5, peakf = 8.0, metric-'rms', window_seconds=10): # Q=None or Q=23

    # same grid as before
    source = initial_source()
    gridobj = make_grid()
     
    # Load inventory
    invMVO = obspy.read_inventory(os.path.join(RESPONSE_DIR,'MV.xml'), format='stationxml')

    # remove instrument response - and set units accordingly IMPORTANT!
    pre_filt = [0.4, 0.5, 18, 20]
    #pre_filt = [6.0, 7.0, 9.0, 10.0]
    for tr in st:
        print(tr)
        try:
            tr.remove_response(inventory=invMVO, pre_filt=pre_filt, output="DISP", plot=True) 
        except:
            st.remove(tr)
        else:
            tr.stats['units'] = 'm'
    #st.plot(equal_scale=True);

    # compute DSAM data with 10-s time window
    dsamObj = DSAM(stream=st, sampling_interval=window_seconds)
    print(dsamObj)
    dsamObj.plot(metrics=metric, equal_scale=True)

    # compute reduced displacement from DSAM data
    DRSobj = dsamObj.compute_reduced_displacement(invMVO, source, surfaceWaves=True, Q=Q, wavespeed_kms=surfaceWaveSpeed_kms, peakf=peakf)

    # plot DRS data - we assume the frequency band of interest for PDCs is same as the VT band we use for band ratio calculations. I think this is 4-18 Hz. But we could give different banks to the DSAM calculation above
    DRSmaxrms = DRSobj.max(metric='VT')
    print(f'Maximum DRS assuming fixed source is: {DRSmaxrms}')
    DRSobj.plot(equal_scale=True)   

    # Create an ASL object with DSAM (displacement amplitude) data, inventory data, and a grid object. The inventory is used for station locations to compute distances
    aslobj = ASL(dsamObj, 'VT', invMVO, gridobj)

    # Compute grid distances
    aslobj.compute_grid_distances()

    # Compute amplitude corrections
    aslobj.compute_amplitude_corrections(surfaceWaves=True, wavespeed_kms=surfaceWaveSpeed_kms, Q=Q, fix_peakf = peakf)

    # Estimate source location for each time window of window_seconds
    source_pf = aslobj.fast_locate()

    # Plot the source location estimates
    aslobj.plot(source_pf, zoom_level=1, threshold_DR=0.03, scale=0.2, join=True, number=10, equal_size=True, add_labels=False) 
                


#read_monty_wavfile_and_correct_traceIDs(wavpath, bool_ASN=False)
startdate = obspy.UTCDateTime(2001,1,1)
enddate = obspy.UTCDateTime(2001,1,5)
nordicfilelist = seisan.get_sfile_list(SEISAN_DATA, DB, startdate, enddate) 
print(nordicfilelist) 
for nfile in nordicfilelist:
    try:
        s = seisan_classes.Sfile(nfile, fast_mode=True)
        d = s.to_dict()
        display(d)
        sfileindex_dict = {'sfile':os.path.basename(s.path), 'DSN_wavfile':None, 'DSN_exists':False, 'ASN_wavfile':None, 'ASN_exists':False, 'corrected_DSN_mseed':None, 'corrected_ASN_mseed':None, 'mainclass':s.mainclass, 'subclass':s.subclass}
    except:
        continue

    for item in ['wavfile1', 'wavfile2']: # there can be up to 2 wavfiles per sfile, One from DSN and one from ASN. But here we are only interested in DSN, and these have 'MVO' in the filename
        wavfile = d[item]
        if wavfile and 'MVO' in os.path.basename(wavfile):
            st = obspy.Stream()
            if os.path.isfile(wavfile):
                #try:
                st = libMVO.read_monty_wavfile_and_correct_traceIDs(wavfile, bool_ASN=False)
                print(st)
                #except:
                #    print(f'could not load {wavfile}')
            else:
                altbase = os.path.basename(wavfile).split('.')[0][0:-3]
                pattern = os.path.join(os.path.dirname(wavfile), altbase+'*')
                similar_wavfiles = glob.glob(pattern)
                if len(similar_wavfiles)==1:
                    wavfile = similar_wavfiles[0]
                    #try:
                    st = libMVO.read_monty_wavfile_and_correct_traceIDs(wavfile, bool_ASN=False)
                    print(st)
                    #except:
                    #    print(f'could not load {wavfile}')    
                else:
                    print(f'got {len(similar_wavfiles)} similar wavfiles matching {pattern}')                
                #print(f'no such file {wavfile}')
            if len(st) > 0:
                asl_event(st)

    #display(sfileindex_dict)                       
                